## THY 1.THY_DATA.txt veri setini kullanarak aşağıdaki görevleri yapınız.

# GÖREV 1: Veriyi Hazırlama

### Adım 1: Başlangıç

- Veriyi HDFS üzerinde datasets klasörü altına taşıyınız.
- Gerekli Kütüphaneleri yükleyiniz.
- Spark session oluşturunuz. Apache Spark'ı YARN modunda kullanınız.

**Not:** Veriyi `inferSchema` **True** olacak sekilde okutunuz, ilgili csv dosyasına göre `delimiter` seçiniz.

## 1-) Veri setini Hadoop HDFS'e yükleyiniz.

In [1]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [6]:
spark = (
    SparkSession.builder 
    .appName("THY Analizi") 
    .master("yarn") 
    .enableHiveSupport() 
    .getOrCreate()
)

In [7]:
df = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("/user/train/datasets/thy_data.txt")

In [8]:
df.limit(5).toPandas()

,SEASON,ORIGIN,DESTINATION,PSGR_COUNT
0,SUMMER,227,YYZ,6
1,SUMMER,224,222,3
2,SUMMER,226,JF8,1
3,SUMMER,227,3RG,11
4,SUMMER,227,6RV,245


In [14]:
# toplam gözlem sayısı

df.count()

1719202

In [16]:
#toplam değişken sayısı

len(df.columns)

4

In [18]:
# Değişken tiplerini inceleyiniz.

df.printSchema()

root
 |-- SEASON: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DESTINATION: string (nullable = true)
 |-- PSGR_COUNT: integer (nullable = true)



# GÖREV 3: Veri Analizi
- Unique Değişken Kontrolü
- Tanımlayıcı İstatistikler 
- Kalkış noktasına göre yolcu sayısı
- Yolcu Sayısının Mevsimlere Göre İncelenmesi

 ### Adım1: Origin değişkenindeki eşsiz sınıf sayısını bulunuz

In [22]:
df.select("ORIGIN").distinct().count()

4210

### Adım2: Mevsimlere göre istatistiklerini inceleyiniz. (sum, avg, count)

In [23]:
from pyspark.sql.functions import sum, avg, count

mevsim_istatistikleri_df = df.groupBy("SEASON").agg(
    sum("PSGR_COUNT").alias("toplam_yolcu"),
    avg("PSGR_COUNT").alias("ortalama_yolcu"),
    count("*").alias("toplam_ucus_sayisi")
)

In [24]:
mevsim_istatistikleri_df.orderBy("SEASON").show()

+------+------------+-----------------+------------------+
|SEASON|toplam_yolcu|   ortalama_yolcu|toplam_ucus_sayisi|
+------+------------+-----------------+------------------+
|SUMMER|   521002675|567.8806552066541|            917451|
|WINTER|   336882094|420.1829420855103|            801751|
+------+------------+-----------------+------------------+



### Adım 3: Yaz sezonunda en çok yolcunun taşındığı ilk 5 kalkış noktasını gözlemleyiniz.

In [25]:
from pyspark.sql.functions import col, sum

summer_flights_df = df.filter(col("SEASON") == "SUMMER") # summer sezonunu filtreliyoruz

top_5_summer_origins = (summer_flights_df
                        .groupBy("ORIGIN")
                        .agg(sum("PSGR_COUNT").alias("toplam_yolcu"))
                        .orderBy(col("toplam_yolcu").desc())
                        .limit(5)
                       )

In [26]:
print("Yaz Sezonunda En Çok Yolcu Taşınan İlk 5 Kalkış Noktası:")
top_5_summer_origins.show()

Yaz Sezonunda En Çok Yolcu Taşınan İlk 5 Kalkış Noktası:
+------+------------+
|ORIGIN|toplam_yolcu|
+------+------------+
|   IC7|    11177363|
|   LHR|     9696224|
|   H8G|     8432456|
|   DEL|     7705173|
|   CDG|     7244943|
+------+------------+



### Adım 4: Veri setini okuyarak mevsim ve kalkış noktasına göre yolcu sayısını azalan olarak sıralayınız

In [27]:
grouped_and_sorted_df = (df
                         .groupBy("SEASON", "ORIGIN")
                         .agg(sum("PSGR_COUNT").alias("toplam_yolcu"))
                         .orderBy(col("toplam_yolcu").desc())
                        )

In [28]:
print("Mevsim ve Kalkış Noktasına Göre Toplam Yolcu Sayıları (Azalan Sırada):")
grouped_and_sorted_df.show()

Mevsim ve Kalkış Noktasına Göre Toplam Yolcu Sayıları (Azalan Sırada):
+------+------+------------+
|SEASON|ORIGIN|toplam_yolcu|
+------+------+------------+
|SUMMER|   IC7|    11177363|
|SUMMER|   LHR|     9696224|
|SUMMER|   H8G|     8432456|
|WINTER|   IC7|     7803139|
|SUMMER|   DEL|     7705173|
|SUMMER|   CDG|     7244943|
|SUMMER|   488|     7216564|
|SUMMER|   JF8|     6304421|
|WINTER|   H8G|     6044541|
|WINTER|   LHR|     5954850|
|WINTER|   488|     5904370|
|SUMMER|   4OM|     5852877|
|SUMMER|   6PE|     5797686|
|SUMMER|   L2X|     5696390|
|SUMMER|   DX4|     5682622|
|SUMMER|   PVG|     5569012|
|WINTER|   DEL|     5504620|
|SUMMER|   M2D|     5076337|
|SUMMER|   2M3|     5072853|
|SUMMER|   3I7|     4984993|
+------+------+------------+
only showing top 20 rows



# GÖREV 4: Veri Kaynaklarına Yazdırma

#### Hive
- Elde ettiğiniz veri setini hive veri tabanına orc formatında yazınız.

#### Veri Okuma

- Hive'a gönderdiğiniz veri setini okuyunuz.

#### Postgresql
- Elde ettiğiniz veri setini Postgresql'e traindb.public.ordered_df_thy tablosuna yazsın.

#### Veri Okuma

- Shell üzerinden Postgresql'e gönderdiğiniz veriyi okuyunuz.

### Adım1: Elde ettiğiniz veri setini Hive üzerinde test1 veri tabanına ORC formatında yazınız

In [37]:
grouped_and_sorted_df.write.format("orc").mode("overwrite").saveAsTable("test1.thy_market_grpby_season_org")

### Adım2: Hive’a gönderdiğiniz veri setini okuyunuz ve ilk 5 gözlemi inceleyiniz.

In [38]:
spark.sql("select * from test1.thy_market_grpby_season_org;").limit(5).toPandas()

,SEASON,ORIGIN,toplam_yolcu
0,WINTER,YLP,1
1,WINTER,3VI,1
2,WINTER,4YD,1
3,WINTER,34R,1
4,SUMMER,47W,1


### Adım3: Elde ettiğiniz veri setini Postgresql üzerinde traindb veri tabanı içerisinde public.ordered_df_thy tablosuna yazınız

In [39]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [41]:
grouped_and_sorted_df.write \
.jdbc(url=jdbcUrl,
              table="grouped_and_sorted_df", 
              mode="overwrite", 
              properties={"driver": 'org.postgresql.Driver'})

### Adım 4: Shell üzerinden Postgresqle gönderdiğiniz veriyi okuyunuz.

In [43]:
spark.read.jdbc(url=jdbcUrl,
              table="grouped_and_sorted_df",  
              properties={"driver": 'org.postgresql.Driver'}).limit(5).toPandas()

,SEASON,ORIGIN,toplam_yolcu
0,SUMMER,4C7,3578345
1,SUMMER,IC7,11177363
2,SUMMER,JED,3571994
3,SUMMER,LHR,9696224
4,WINTER,L2X,3347347
